# Corona analysis, infection rate, hospital bed and death correlation etc
Read data from wikipedia, worldometer and the EU

In [5]:
#Libraries
import pandas as pd
import numpy as np
import plotly.express as px

In [6]:
#Hospital beds, copy from wikipedia: https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds
hospital_beds = pd.read_excel(r'C:\Users\...\hospitalbeds.xlsx',sheet_name='hospital_beds')

#Download from EU: https://data.europa.eu/euodp/en/data/dataset/covid-19-coronavirus-data
url="https://opendata.ecdc.europa.eu/covid19/casedistribution/csv"
EU_data=pd.read_csv(url)

In [7]:
hospital_beds.drop(['Rank','Ventilators'],axis=1,inplace=True)
hospital_beds.columns = ['Country','Continent','Hospital_beds_per100_2017','ICU_beds_per100k','Federal']
hospital_beds

,Country,Continent,Hospital_beds_per100_2017,ICU_beds_per100k,Federal
0,Japan,Asia,13.05,7.3,no
1,South Korea,Asia,12.27,10.6,no
2,Russia,Europe,8.05,8.3,yes
3,Germany,Europe,8.00,29.2,yes
4,Austria,Europe,7.37,21.8,yes
5,Hungary,Europe,7.02,13.8,no
6,Czech Republic,Europe,6.63,11.6,no
7,Poland,Europe,6.62,6.9,no
8,Lithuania,Europe,6.56,15.5,no
9,France,Europe,5.98,11.6,no


In [8]:
EU_data.rename({'countriesAndTerritories':'Country','dateRep':'Date'},axis=1,inplace=True)
#EU_data.drop(['geoId','countryterritoryCode'],axis=1,inplace=True)
EU_data['Date']=pd.to_datetime(EU_data[['year','month','day']])
#EU_data['popData2018']=pd.to_numeric(EU_data['popData2018'],downcast='integer',errors='coerce')
EU_data

,Date,day,month,year,cases,deaths,geoId,continentExp,countryterritoryCode,popData2018,Country
0,2020-04-20,20,4,2020,88,3,AF,Asia,AFG,37172386.0,Afghanistan
1,2020-04-19,19,4,2020,63,0,AF,Asia,AFG,37172386.0,Afghanistan
2,2020-04-18,18,4,2020,51,1,AF,Asia,AFG,37172386.0,Afghanistan
3,2020-04-17,17,4,2020,10,4,AF,Asia,AFG,37172386.0,Afghanistan
4,2020-04-16,16,4,2020,70,2,AF,Asia,AFG,37172386.0,Afghanistan
...,...,...,...,...,...,...,...,...,...,...,...
11976,2020-03-25,25,3,2020,0,0,ZW,Africa,ZWE,14439018.0,Zimbabwe
11977,2020-03-24,24,3,2020,0,1,ZW,Africa,ZWE,14439018.0,Zimbabwe
11978,2020-03-23,23,3,2020,0,0,ZW,Africa,ZWE,14439018.0,Zimbabwe
11979,2020-03-22,22,3,2020,1,0,ZW,Africa,ZWE,14439018.0,Zimbabwe


In [9]:
#Rename country names to align to each other
hospital_beds['Country'].replace(regex={'South Korea':'South_Korea','Czech Republic':'Czech_Republic','Hong Kong':'Hong_Kong',
                                 'United States':'USA','New Zealand':'New_Zealand','United Kingdom(more)':'UK'},inplace=True)
hospital_beds['Country'].replace({'United Kingdom(more)':'UK'},inplace=True)

EU_data['Country'].replace(regex={'Czechia':'Czech_Republic','United_States_of_America':'USA','United_Kingdom':'UK'},inplace=True)

# Time Series data

In [11]:
#EU_data.groupby(['Date','Country']).cumsum()
Cumulative_deaths = EU_data.groupby(['Country','Date','popData2018'])['deaths'].sum().groupby(level=0).cumsum().reset_index()
Cumulative_deaths.sort_values(by=['Country','Date'],inplace=True)
Cumulative_deaths['Dead_per_million']=Cumulative_deaths['deaths']/Cumulative_deaths['popData2018']*1000000
#Cumulative_deaths[(Cumulative_deaths['Country']=='Germany') & (Cumulative_deaths['Dead_per_million']>0)]#.sort_values(by=['Date'],ascending=True)

In [12]:
#Drop all deaths per million under 1
Cumulative_deaths.drop(Cumulative_deaths[Cumulative_deaths.Dead_per_million < 1].index, inplace=True)
#To align all countries for time we need to count days starting from the first day, there were more than 1 death per million
Cumulative_deaths['Day']=Cumulative_deaths.groupby(['Country'])['Date'].cumcount(ascending=True)+1
#Cumulative_deaths[Cumulative_deaths['Country']=='Japan']

In [18]:
#Figure showing death rates
fig = px.scatter(Cumulative_deaths, x="Day", y="Dead_per_million", 
                 color='Country',
                 color_discrete_sequence=px.colors.qualitative.Dark24,
                 log_y=True,
                 #text="Country",
                 size='popData2018')

fig.update_layout(
    title_text='Cumulative Covid-19 deaths (bubble size = population size)',
    xaxis_title="Number of days since at least 1 death per million of population",
    yaxis_title="Log of deaths due to Covid-19 per million population"
)
    
fig.show()

In [16]:
#Merge to hospital data set

#Wouldn't merge, so checked the encoding and had to replace following instructions form
#https://stackoverflow.com/questions/39582984/pandas-merging-on-string-columns-not-working-bug
hospital_beds.Country = hospital_beds.Country.str.replace('\xa0','')

Cumulative_deaths = Cumulative_deaths.merge(hospital_beds, on='Country',how='left')
#Cumulative_deaths.dropna()

In [20]:
#Animation for each day

fig = px.scatter(Cumulative_deaths.dropna(), x="Hospital_beds_per100_2017", y="Dead_per_million", 
                 animation_frame="Day", animation_group="Country",
           size="popData2018", color="Continent", hover_name="Country",
           #log_y=True, 
           size_max=55, range_x=[0,14], range_y=[1,500])#2400])

fig.update_layout(
    title_text='Covid-19 deaths per million vs. number of hospital beds (bubble size = population size)',
    xaxis_title="Hospital beds per 1000 people in 2017 (Wikipedia)",
    yaxis_title="Deaths due to Covid-19 per million population"
)

fig.show()